# Resampling Methods


[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/lee-t/ISL-ch5-jupyterlite/HEAD?urlpath=%2Fdoc%2Ftree%2Fcontent%2FCh05-resample-lab.ipynb) 
Resampling methods are an indispensable tool in modern statistics. They
involve repeatedly drawing samples from a training set and refitting a model
of interest on each sample in order to obtain additional information about
the fitted model. For example, in order to estimate the variability of a linear
regression fit, we can repeatedly draw different samples from the training
data, fit a linear regression to each new sample, and then examine the
extent to which the resulting fits differ. Such an approach may allow us to
obtain information that would not be available from fitting the model only
once using the original training sample.

In this chapter, we discuss two of the most commonly
used resampling methods, *cross-validation* and the *bootstrap*.

Cross-validation is most often used to estimate test error associated with a statistical learning method, whereas the bootstrap is most commonly used to provide a measure of accuracy for a given parameter/method.

The process
of evaluating a model's performance is known as *model assessment*, whereas 
the process of selecting the proper level of flexibility for a model is known as *model selection*.

## Setup

Import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ISLP import load_data
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Set plot style
sns.set_theme(style='whitegrid')
%matplotlib inline

## Cross Validation

Sometimes we want to estimate the test error rate using the available training data.
A number of approaches can be used for this.
In this section we consider methods which involve *holding out* a subset of the training data from the fitting process, then applying the model to that hold-out set for model assessment.

### The Validation Set Approach

This simple strategy involves randomly dividing available observations into training and validation sets.
The model is fit on the training set, and used to make predictions on the validation set.
The corresponding metric from the validation set predictions -- usually MSE in the case of a quantitative response -- provides an estimate of the test error rate.

Load the `Auto` data set:

In [ ]:
# Load the Auto dataset
Auto = load_data('Auto')
Auto.head()

Randomly split the data into 50% training and 50% validation, fit on the training set, and compute the MSE on the validation set.
We'll repeat this 10 times to reproduce a figure similar to Figure 5.2.

In [ ]:
def evaluate_poly_models(X_train, X_test, y_train, y_test, max_degree=10):
    """Evaluate polynomial models of different degrees."""
    mse_values = []
    for degree in range(1, max_degree + 1):
        # Create polynomial features
        poly_features = PolynomialFeatures(degree=degree, include_bias=False)
        X_train_poly = poly_features.fit_transform(X_train.reshape(-1, 1))
        X_test_poly = poly_features.transform(X_test.reshape(-1, 1))
        
        # Fit linear regression
        model = LinearRegression()
        model.fit(X_train_poly, y_train)
        
        # Predict and calculate MSE
        y_pred = model.predict(X_test_poly)
        mse = mean_squared_error(y_test, y_pred)
        mse_values.append(mse)
    
    return mse_values

# Set seed for reproducibility
np.random.seed(10)

# Prepare data
X = Auto['horsepower'].values
y = Auto['mpg'].values

# Single split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=10)
mse_single = evaluate_poly_models(X_train, X_test, y_train, y_test)

print("MSE for different polynomial degrees (single split):")
for i, mse in enumerate(mse_single[:3], 1):
    print(f"Degree {i}: {mse:.2f}")

In [ ]:
# Repeat validation split approach 10 times
all_mse = {}
for i in range(1, 11):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=i)
    mse_values = evaluate_poly_models(X_train, X_test, y_train, y_test)
    all_mse[i] = mse_values

# Plot results (similar to Figure 5.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: single split
degrees = range(1, 11)
ax1.plot(degrees, all_mse[1], marker='o', linewidth=2, markersize=8)
ax1.set_xlabel('Degree of polynomial')
ax1.set_ylabel('MSE')
ax1.set_ylim([15, 30])
ax1.set_title('Single Validation Split')
ax1.grid(True)

# Right panel: multiple splits
colors = plt.cm.tab10(np.linspace(0, 1, 10))
for i, color in enumerate(all_mse.keys(), 1):
    ax2.plot(degrees, all_mse[i], linewidth=1.5, alpha=0.7, color=colors[i-1])
ax2.set_xlabel('Degree of polynomial')
ax2.set_ylabel('MSE')
ax2.set_ylim([15, 30])
ax2.set_title('Multiple Validation Splits')
ax2.grid(True)

plt.tight_layout()
plt.show()

As is clear from the right-hand panel, this approach is highly variable depending on the testing/validation set split.
Another downside is that, because the training set used to fit the data has fewer observations, it tends to overestimate the test error rate on the entire data set.

### Leave-One-Out Cross Validation (LOOCV)

*Leave-one-out cross validation* (LOOCV) attempts to address the shortcomings of the validation set approach.
It still involves splitting the $n$ observations into two parts, but it repeats it $n$ times, with a single observation $(x_i, y_i)$ as the hold-out "set" and the remaining $n-1$ observations as the training set.
The MSE for each iteration is simply $\text{MSE}_i = (y_i - \hat{y}_i)^2$.
Then the LOOCV estimate of the MSE is the average over all observations:

$$
\text{CV}_{(n)} = \frac{1}{n} \sum_{i=1}^n \text{MSE}_i.
$$

The LOOCV approach has advantages over the validation set approach:
- First, it has far less bias since we use training sets with $n - 1$ observations.
- Second, it yields deterministic results (no randomness in splits).

In [ ]:
# LOOCV for polynomial regression
from sklearn.model_selection import LeaveOneOut

def loocv_mse(X, y, degree):
    """Calculate LOOCV MSE for a polynomial model."""
    loo = LeaveOneOut()
    mse_scores = []
    
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly_features.fit_transform(X.reshape(-1, 1))
    
    for train_idx, test_idx in loo.split(X_poly):
        X_train, X_test = X_poly[train_idx], X_poly[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_scores.append((y_test[0] - y_pred[0])**2)
    
    return np.mean(mse_scores)

# Calculate LOOCV MSE for degrees 1-10
# Note: This can be slow for large datasets
print("Computing LOOCV MSE (this may take a moment)...")
loocv_mse_values = [loocv_mse(X, y, degree) for degree in range(1, 11)]

print("\nLOOCV MSE for different polynomial degrees:")
for i, mse in enumerate(loocv_mse_values, 1):
    print(f"Degree {i}: {mse:.2f}")

### $k$-fold Cross-Validation

*$k$-fold CV* involves randomly dividing the observations into $k$ groups/folds of approximately equal size.
The first fold is used as the validation/assessment set, and the remaining $k-1$ folds used to fit the model.
This is repeated $k$ times, with each fold being used as the assessment set once.
The $k$-fold CV estimate of the test error is then the average:

$$
\text{CV}_{(k)} = \frac{1}{k} \sum_{i=1}^k \text{MSE}_i.
$$

LOOCV is a special case of $k$-fold CV where $k = n$. In practice, one typically performs $k$-fold CV using $k$ = 5
or $k$ = 10. The advantage is computational: LOOCV requires fitting the model $n$ times, while 10-fold
CV requires only ten fits.

In [ ]:
# 10-fold cross-validation
def kfold_cv_mse(X, y, degree, k=10, random_state=10):
    """Calculate k-fold CV MSE for a polynomial model."""
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    mse_scores = []
    
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly_features.fit_transform(X.reshape(-1, 1))
    
    for train_idx, test_idx in kf.split(X_poly):
        X_train, X_test = X_poly[train_idx], X_poly[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_scores.append(mean_squared_error(y_test, y_pred))
    
    return np.mean(mse_scores)

# Calculate 10-fold CV MSE for degrees 1-10
kfold_mse_values = [kfold_cv_mse(X, y, degree) for degree in range(1, 11)]

print("10-fold CV MSE for different polynomial degrees:")
for i, mse in enumerate(kfold_mse_values, 1):
    print(f"Degree {i}: {mse:.2f}")

In [ ]:
# Repeat 10-fold CV with different random seeds and plot
all_kfold_mse = {}
for seed in range(1, 11):
    mse_values = [kfold_cv_mse(X, y, degree, random_state=seed) for degree in range(1, 11)]
    all_kfold_mse[seed] = mse_values

# Plot results (similar to Figure 5.4)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: LOOCV (deterministic)
degrees = range(1, 11)
ax1.plot(degrees, loocv_mse_values, marker='o', linewidth=2, markersize=8)
ax1.set_xlabel('Degree of polynomial')
ax1.set_ylabel('MSE')
ax1.set_ylim([15, 30])
ax1.set_title('LOOCV')
ax1.grid(True)

# Right panel: 10-fold CV (multiple runs)
degrees = range(1, 11)
colors = plt.cm.tab10(np.linspace(0, 1, 10))
for i, color in enumerate(all_kfold_mse.keys(), 1):
    ax2.plot(degrees, all_kfold_mse[i], linewidth=1.5, alpha=0.7, color=colors[i-1])
ax2.set_xlabel('Degree of polynomial')
ax2.set_ylabel('MSE')
ax2.set_ylim([15, 30])
ax2.set_title('10-fold CV (Multiple Runs)')
ax2.grid(True)

plt.tight_layout()
plt.show()

The $k$-fold CV approach (right panel) still has some variability due to random splitting, but much less than the validation set approach.

## The Bootstrap

The *bootstrap* is a widely applicable and extremely powerful statistical tool that can be used to quantify the uncertainty associated with a given estimator or statistical learning method. As a simple example, the bootstrap can be used to estimate the standard errors of the coefficients from a linear regression fit.

The toy example in this section is about investment in two assets $X$ and $Y$.
We wish to choose a fraction $\alpha$ of investment into $X$ which minimizes the total variance (risk) of the investment.
It can be shown that the optimal value is given by:

$$
\alpha = \frac{\sigma_Y^2 - \sigma_{XY}}{\sigma_X^2 + \sigma_Y^2 - 2\sigma_{XY}},
$$

where $\sigma_X^2 = \text{Var}(X)$, $\sigma_Y^2 = \text{Var}(Y)$, and $\sigma_{XY} = \text{Cov}(X, Y)$.

In [ ]:
# Load Portfolio data
Portfolio = load_data('Portfolio')
Portfolio.head()

In [ ]:
# Function to compute alpha
def alpha_func(X, Y):
    """Compute the optimal alpha for portfolio allocation."""
    return ((np.var(Y) - np.cov(X, Y)[0, 1]) / 
            (np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0, 1]))

# Compute alpha from the full dataset
alpha_hat = alpha_func(Portfolio['X'], Portfolio['Y'])
print(f"Estimated alpha: {alpha_hat:.3f}")

In [ ]:
# Bootstrap to estimate standard error of alpha
def bootstrap_alpha(data, n_bootstrap=1000, random_state=319):
    """Perform bootstrap resampling to estimate SE of alpha."""
    np.random.seed(random_state)
    n = len(data)
    alpha_estimates = []
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        idx = np.random.choice(n, size=n, replace=True)
        X_boot = data['X'].iloc[idx]
        Y_boot = data['Y'].iloc[idx]
        alpha_estimates.append(alpha_func(X_boot, Y_boot))
    
    return np.array(alpha_estimates)

# Perform bootstrap
alpha_boot = bootstrap_alpha(Portfolio, n_bootstrap=1000)

print(f"Bootstrap mean of alpha: {np.mean(alpha_boot):.3f}")
print(f"Bootstrap SE of alpha: {np.std(alpha_boot):.3f}")

In [ ]:
# Visualize bootstrap distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(alpha_boot, bins=30, edgecolor='black', alpha=0.7)
ax1.axvline(alpha_hat, color='red', linestyle='--', linewidth=2, label=f'Original estimate: {alpha_hat:.3f}')
ax1.set_xlabel('Alpha')
ax1.set_ylabel('Frequency')
ax1.set_title('Bootstrap Distribution of Alpha')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Boxplot
ax2.boxplot(alpha_boot, vert=True)
ax2.axhline(alpha_hat, color='red', linestyle='--', linewidth=2)
ax2.set_ylabel('Alpha')
ax2.set_title('Bootstrap Distribution (Boxplot)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Estimating the Accuracy of a Linear Regression Model

The bootstrap approach can be used to assess the variability of the coefficient estimates from a statistical learning method. Here we use the bootstrap to assess the variability of the estimates for $\beta_0$ and $\beta_1$, the intercept and slope terms for the linear regression model that uses horsepower to predict mpg in the `Auto` data set.

In [ ]:
# Fit linear regression model
from sklearn.linear_model import LinearRegression

X_auto = Auto[['horsepower']]
y_auto = Auto['mpg']

model = LinearRegression()
model.fit(X_auto, y_auto)

print(f"Intercept: {model.intercept_:.4f}")
print(f"Coefficient: {model.coef_[0]:.4f}")

In [ ]:
# Bootstrap for regression coefficients
def bootstrap_regression(X, y, n_bootstrap=1000, random_state=42):
    """Perform bootstrap to estimate SE of regression coefficients."""
    np.random.seed(random_state)
    n = len(X)
    intercepts = []
    coefs = []
    
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, size=n, replace=True)
        X_boot = X.iloc[idx]
        y_boot = y.iloc[idx]
        
        model_boot = LinearRegression()
        model_boot.fit(X_boot, y_boot)
        
        intercepts.append(model_boot.intercept_)
        coefs.append(model_boot.coef_[0])
    
    return np.array(intercepts), np.array(coefs)

# Perform bootstrap
boot_intercepts, boot_coefs = bootstrap_regression(X_auto, y_auto)

print("Bootstrap estimates:")
print(f"Intercept - Mean: {np.mean(boot_intercepts):.4f}, SE: {np.std(boot_intercepts):.4f}")
print(f"Coefficient - Mean: {np.mean(boot_coefs):.4f}, SE: {np.std(boot_coefs):.4f}")

### Quadratic Model Bootstrap

We can find better correspondence between bootstrap and regression estimates if we use the quadratic model because it better fits the data:

In [ ]:
# Bootstrap for quadratic regression
def bootstrap_quad_regression(X, y, n_bootstrap=1000, random_state=42):
    """Perform bootstrap for quadratic regression."""
    np.random.seed(random_state)
    n = len(X)
    coef_results = []
    
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, size=n, replace=True)
        X_boot = X.iloc[idx]
        y_boot = y.iloc[idx]
        
        # Create quadratic features
        poly = PolynomialFeatures(degree=2, include_bias=True)
        X_boot_poly = poly.fit_transform(X_boot)
        
        model_boot = LinearRegression(fit_intercept=False)
        model_boot.fit(X_boot_poly, y_boot)
        
        coef_results.append(model_boot.coef_)
    
    return np.array(coef_results)

# Perform bootstrap for quadratic model
boot_quad_coefs = bootstrap_quad_regression(X_auto, y_auto)

print("Bootstrap estimates for quadratic model:")
print(f"Intercept - Mean: {np.mean(boot_quad_coefs[:, 0]):.4f}, SE: {np.std(boot_quad_coefs[:, 0]):.4f}")
print(f"Linear term - Mean: {np.mean(boot_quad_coefs[:, 1]):.4f}, SE: {np.std(boot_quad_coefs[:, 1]):.4f}")
print(f"Quadratic term - Mean: {np.mean(boot_quad_coefs[:, 2]):.4f}, SE: {np.std(boot_quad_coefs[:, 2]):.4f}")

## Summary

In this lab, we explored:
- **Validation Set Approach**: Simple but highly variable
- **LOOCV**: Less bias but computationally expensive
- **k-fold Cross-Validation**: Good balance of bias-variance tradeoff
- **Bootstrap**: Powerful tool for estimating standard errors and uncertainty

These resampling methods are fundamental tools in statistical learning for model assessment and selection.